<a href="https://colab.research.google.com/github/chyj0805/Project_2/blob/main/Fetures/Vantage_API_data_fetches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Fetches

In [4]:
!pip install python-dotenv

In [5]:
import os
from dotenv import load_dotenv

# Load the environment variable file
load_dotenv()

# Get the API key
api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

if api_key is None:
    raise ValueError("Alpha Vantage API key is not set in the environment.")

In [6]:
import requests
import pandas as pd
import os

def get_stock_historical_data(api_key, stock_symbol):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": stock_symbol,
        "apikey": api_key,
        "outputsize": "full",
        "datatype": "json"
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    print(data)  # Debug line

    if "Time Series (Daily)" not in data:
        print("Error fetching data. Check your API key and stock symbol.")
        print(data)
        return

    time_series = data["Time Series (Daily)"]
    data_list = []

    for date, details in time_series.items():
        row = {
            "Date": date,
            "Open": details["1. open"],
            "High": details["2. high"],
            "Low": details["3. low"],
            "Close": details["4. close"],
            "Volume": details["5. volume"]
        }
        data_list.append(row)

    df = pd.DataFrame(data_list)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values(by="Date")

    csv_filename = f"{stock_symbol}_historical_data.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
if api_key is None:
    raise ValueError("Alpha Vantage API key is not set in the environment.")

stock_symbol = "IYE"
get_stock_historical_data(api_key, stock_symbol)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'IYE', '3. Last Refreshed': '2023-08-31', '4. Output Size': 'Full size', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2023-08-31': {'1. open': '46.9100', '2. high': '46.9700', '3. low': '46.5700', '4. close': '46.8500', '5. volume': '174624'}, '2023-08-30': {'1. open': '46.7300', '2. high': '46.9150', '3. low': '46.6100', '4. close': '46.8100', '5. volume': '181727'}, '2023-08-29': {'1. open': '46.4800', '2. high': '46.6125', '3. low': '46.0700', '4. close': '46.5600', '5. volume': '386931'}, '2023-08-28': {'1. open': '46.2100', '2. high': '46.7200', '3. low': '46.1140', '4. close': '46.3900', '5. volume': '361019'}, '2023-08-25': {'1. open': '45.8800', '2. high': '46.2900', '3. low': '45.5100', '4. close': '46.0500', '5. volume': '455569'}, '2023-08-24': {'1. open': '45.6100', '2. high': '46.1000', '3. low': '45.5500', '4. close': '45.5900', '5. volume': '417455'}, '2023-08-23

In [7]:
def get_MACD_data(api_key, stock_symbol, interval="daily", series_type="close"):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "MACD",
        "symbol": stock_symbol,
        "interval": interval,
        "series_type": series_type,
        "apikey": api_key,
        "datatype": "json"
    }

    # Make the API request and fetch the data
    response = requests.get(base_url, params=params)
    data = response.json()

    # Debug line to print the data
    print(data)

    # Check if the MACD data is present in the response
    if "Technical Analysis: MACD" not in data:
        print("Error fetching MACD data. Check your API key and stock symbol.")
        print(data)
        return

    # Extract and process the MACD data
    macd_series = data["Technical Analysis: MACD"]
    macd_list = []
    for date, details in macd_series.items():
        row = {
            "Date": date,
            "MACD_Signal": details["MACD_Signal"],
            "MACD_Hist": details["MACD_Hist"],
            "MACD": details["MACD"]
        }
        macd_list.append(row)

    # Convert the list of MACD data to a DataFrame
    df = pd.DataFrame(macd_list)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values(by="Date")

    # Save the MACD data to a CSV file
    csv_filename = f"{stock_symbol}_MACD_data.csv"
    df.to_csv(csv_filename, index=False)
    print(f"MACD data saved to {csv_filename}")

# Check if the API key is set in the environment
api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
if api_key is None:
    raise ValueError("Alpha Vantage API key is not set in the environment.")

# Define the stock symbol for which to fetch the MACD data
stock_symbol = "IYE"

# Call the function to fetch and save the MACD data
get_MACD_data(api_key, stock_symbol)

{'Meta Data': {'1: Symbol': 'IYE', '2: Indicator': 'Moving Average Convergence/Divergence (MACD)', '3: Last Refreshed': '2023-08-31', '4: Interval': 'daily', '5.1: Fast Period': 12, '5.2: Slow Period': 26, '5.3: Signal Period': 9, '6: Series Type': 'close', '7: Time Zone': 'US/Eastern'}, 'Technical Analysis: MACD': {'2023-08-31': {'MACD': '0.4355', 'MACD_Signal': '0.5000', 'MACD_Hist': '-0.0645'}, '2023-08-30': {'MACD': '0.4244', 'MACD_Signal': '0.5161', 'MACD_Hist': '-0.0917'}, '2023-08-29': {'MACD': '0.4081', 'MACD_Signal': '0.5390', 'MACD_Hist': '-0.1309'}, '2023-08-28': {'MACD': '0.4068', 'MACD_Signal': '0.5717', 'MACD_Hist': '-0.1649'}, '2023-08-25': {'MACD': '0.4166', 'MACD_Signal': '0.6129', 'MACD_Hist': '-0.1964'}, '2023-08-24': {'MACD': '0.4576', 'MACD_Signal': '0.6620', 'MACD_Hist': '-0.2044'}, '2023-08-23': {'MACD': '0.5502', 'MACD_Signal': '0.7131', 'MACD_Hist': '-0.1630'}, '2023-08-22': {'MACD': '0.6185', 'MACD_Signal': '0.7539', 'MACD_Hist': '-0.1353'}, '2023-08-21': {'MA

In [8]:
data = pd.read_csv("IYE_historical_data.csv")
data

,Date,Open,High,Low,Close,Volume
0,2000-06-16,51.61,51.7800,51.550,51.77,2000
1,2000-06-19,51.53,51.5300,50.630,50.63,2400
2,2000-06-20,50.11,50.1100,49.780,49.78,1000
3,2000-06-21,50.44,51.7000,50.440,51.67,3200
4,2000-06-22,51.92,51.9200,50.520,50.53,2100
...,...,...,...,...,...,...
5834,2023-08-25,45.88,46.2900,45.510,46.05,455569
5835,2023-08-28,46.21,46.7200,46.114,46.39,361019
5836,2023-08-29,46.48,46.6125,46.070,46.56,386931
5837,2023-08-30,46.73,46.9150,46.610,46.81,181727


# ydata-profilling

In [10]:
pip install ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 34.1 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=5892b36661a7542e3602891b331a928a51340f5dd3d77c190d515cfbcb17a026
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.2.1
    Uninstalling pydantic-2.2.1:
      Successfully uninstalled pydantic-2.2.1


In [11]:
from ydata_profiling import ProfileReport

profile = ProfileReport(data, title="IYE Historical Data Profiling Report")

# to HTML
profile.to_file("IYE_historical_data_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]